<a href="https://colab.research.google.com/github/1HarishK1/NLTK_processing-/blob/main/Movie_Recommendations_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Movie=pd.read_csv("/content/drive/MyDrive/imdb_top_1000.csv")

In [ ]:
Movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


In [ ]:
Movie.rename(columns = {'Series_Title':'Title'},inplace='true')

In [ ]:
Movie['Title']=Movie['Title'].apply(str.lower)

In [ ]:
Movie.fillna("")
Movie.isnull().sum()

Poster_Link        0
Title              0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

In [ ]:
Movie.duplicated().sum()

0

In [ ]:
Movie['Star1']

0            Tim Robbins
1          Marlon Brando
2         Christian Bale
3              Al Pacino
4            Henry Fonda
             ...        
995       Audrey Hepburn
996     Elizabeth Taylor
997       Burt Lancaster
998    Tallulah Bankhead
999         Robert Donat
Name: Star1, Length: 1000, dtype: object

In [ ]:
Movie['Star1']=Movie['Star1'].apply(lambda x:x.replace(" ",""))
Movie['Star2']=Movie['Star2'].apply(lambda x:x.replace(" ",""))
Movie['Star3']=Movie['Star3'].apply(lambda x:x.replace(" ",""))
Movie['Star4']=Movie['Star4'].apply(lambda x:x.replace(" ",""))
Movie['Director']=Movie['Director'].apply(lambda x:x.replace(" ",""))

In [ ]:
Movie['Director'].head (25)

0          FrankDarabont
1     FrancisFordCoppola
2       ChristopherNolan
3     FrancisFordCoppola
4            SidneyLumet
5           PeterJackson
6       QuentinTarantino
7        StevenSpielberg
8       ChristopherNolan
9           DavidFincher
10          PeterJackson
11        RobertZemeckis
12           SergioLeone
13          PeterJackson
14         LanaWachowski
15        MartinScorsese
16         IrvinKershner
17           MilosForman
18            ThomasKail
19            BongJoonHo
20          SudhaKongara
21      ChristopherNolan
22     FernandoMeirelles
23         HayaoMiyazaki
24       StevenSpielberg
Name: Director, dtype: object

In [ ]:
Movie['Stars']=Movie[['Star1','Star2','Star3','Star4']].apply(" ".join,axis=1)
Movie['Stars'][0]

'TimRobbins MorganFreeman BobGunton WilliamSadler'

In [ ]:
Movie.Overview=Movie.Overview.apply(lambda x:x.replace('.',''))
Movie.Overview=Movie.Overview.apply(lambda x:x.split())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for x in text.split( ):
     y.append(ps.stem(x))
  return " ".join(y)

In [ ]:
Movie['Overview']=Movie['Overview'].apply(stem)

In [ ]:
Movie['tag']=Movie[['Title','Genre', 'Overview', 'Director','Stars']].astype(str).apply(" ".join,axis=1)

In [ ]:
Movie['tag']=Movie['tag'].apply(str.lower).tolist()

In [ ]:
Movie['tag'][0]

"the shawshank redemption Drama ['Two', 'imprisoned', 'men', 'bond', 'over', 'a', 'number', 'of', 'years,', 'finding', 'solace', 'and', 'eventual', 'redemption', 'through', 'acts', 'of', 'common', 'decency'] FrankDarabont TimRobbins MorganFreeman BobGunton WilliamSadler"

Converting text data to numerica vectors

In [ ]:
cv=CountVectorizer()
designed_vectors=cv.fit_transform(Movie['tag']).toarray()

In [ ]:
designed_vectors.shape

(1000, 9886)

In [ ]:
len(cv.get_feature_names())

AttributeError: ignored

In [ ]:
cos_similarity=cosine_similarity(designed_vectors)
cos_similarity.shape

(1000, 1000)

In [ ]:
sorted(list(enumerate(cos_similarity[0])),reverse=True,key=lambda x:x[1])[1:10]

[(6, 0.37073899893266327),
 (773, 0.33351867298253507),
 (268, 0.31750031750047625),
 (13, 0.3135447206678483),
 (5, 0.3135447206678482),
 (11, 0.31318038399724624),
 (189, 0.3095103817997673),
 (821, 0.3092609442453742),
 (411, 0.3039740428042212)]

Users Could search for below mentioned items.

In [ ]:
def movie_recommend(movie):
  movie_index=Movie[Movie.Title==movie].index[0]
  distance=cos_similarity[movie_index]
  movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:10]

  for x in movie_list:
      print(Movie.iloc[x[0]].Title)

In [ ]:
movie_recommend('alien')

the thing
invasion of the body snatchers
the day the earth stood still
misery
shaun of the dead
star trek ii: the wrath of khan
abre los ojos
aliens
raging bull
